In [3]:
import csv
import random
import time
from IPython.display import clear_output

def import_test_bank():
    """
    Imports a csv test bank, turns each row into an element of a list,
    and removes the first key row
    """
    with open("CPSC 236 TestBank.csv","r") as file:
        reader=csv.reader(file)
        bank=[]
        for row in reader:
            bank.append(row)
        bank.pop(0)
    return bank

def get_question(test_bank,used_up):
    """
    Accepts the question bank list and a list of question indexes 
    already selected, returns a list contianing a randomly selected 
    question and an updated version of the used index list
    """
    while True:
        question_num=random.randint(0,len(test_bank)-1)
        if question_num in used_up:
            print("[Skipping duplicate question]")
            continue
        used_up.append(question_num)
        question_list=test_bank[question_num]
        return question_list,used_up

def display_question(question_info,question_number):
    """
    Isolated code to display the active question in the console.
    """
    print("Question #"+str(question_number)+": "+question_info[0])
    print("A: "+question_info[1])
    print("B: "+question_info[2])
    print("C: "+question_info[3])

def get_user_info():
    """
    Gets user information. Exits with a break code if the user
    inputs 3 invalid IDs. Returns a list of the user's first name,
    last name, and ID #
    """
    bad_attempts=0
    break_code="False"
    user_info=[]
    fname_entry=input("First name:\t\t")
    lname_entry=input("Last name:\t\t")
    while bad_attempts<=2:
        id_entry=input("Student ID (A#####):\t")
        if id_entry.startswith("A")==True:
            if len(id_entry)==6:
                break
            else:
                print("Student ID number consists of 'A' followed by 5 numerals.")
                bad_attempts+=1
        else:
            print("Student ID should begin with 'A'")  
            bad_attempts+=1
    if bad_attempts>2:
        break_code="True"
        print("Maximum number of attempts reached, terminating quiz.")
    print()
    user_info.append(fname_entry)
    user_info.append(lname_entry)
    user_info.append(id_entry)
    return user_info,break_code

def write_results(user_info,score,time,used_questions,student_answers,skipped):
    """
    Writes all relevant information to a results .txt file. Generates a filename from user_info
    and writes the student's score, elapsed time, and individual question results to this file.
    The function also accepts and writes a count of questions skipped due to time limits.
    """
    index=0
    filename=user_info[0]+"_"+user_info[1]+"_"+user_info[2]+".txt"
    mintime,sectime=format_seconds(time)
    score_pct=round((score/10)*100,2)
    with open(filename,"w") as file:
        file.write("Student ID: "+user_info[2]+"\n")
        file.write("Name: "+user_info[0]+" "+user_info[1]+"\n\n")
        file.write("Score: "+str(score)+"/10\t("+str(score_pct)+"%)\n")
        file.write("Time elapsed: "+str(mintime)+" minutes and "+str(sectime)+" seconds\n\n")
        for item in used_questions:
            file.write("#"+str(index+1)+": "+item[0]+"\n")
            file.write("Correct answer: "+item[4]+"\n")
            file.write("Student answer: "+student_answers[index]+"\n")
            index+=1
        if skipped>0:
            file.write("\n"+str(skipped)+" questions were skipped due to running out of time.")
        print("Results successfully written to "+filename)

def test_size():
    """
    Gets user input about the length of their quiz, returns the desired cap and
    a weight value to use when grading each question.
    """
    print("How many questions would you like your quiz to have?")
    print("(A) - 10 questions")
    print("(B) - 20 questions")
    while True:
        size_code=input()
        if size_code.upper()=="A":
            print("Proceeding with 10 questions\n")
            test_cap=11
            weight=1
            break
        elif size_code.upper()=="B":
            print("Proceeding with 20 questions\n")
            test_cap=21
            weight=0.5
            break
        else:
            print("Valid inputs are 'A' or 'B'")
    return test_cap,weight

def format_seconds(time):
    """
    Translates a seconds value into minutes and remaining seconds
    """
    minutes=time//60
    seconds=time-(minutes*60)
    return minutes,seconds

def main():
    test_bank=import_test_bank()
    break_code="False"
    # Break code is used to prevent the program from running after the user has triggered it.
    # Only break code in current implementation comes from get_user_info
    while break_code=="False":
        #Initialize needed default values and lists.
        question_number=1
        correct_ct=0
        ttime=0
        used_up=[]
        used_questions=[]
        student_answers=[]
        skipped=0
        break_code=False
        user_info,break_code=get_user_info()
        # Test begins now. Starting time is fetched and stored.
        start_time=round(time.time(),0)
        while break_code=="False":
            test_cap,weight=test_size()
            while question_number<test_cap:
                #Test begins to loop. 
                question_info,used_up=get_question(test_bank,used_up)
                display_question(question_info,question_number)
                correct_answer=question_info[4]
                while True:
                    # User enters a response. Loops until a valid A,B,C is entered.
                    user_answer=input("Answer:\t")
                    if user_answer.upper()=="A" or user_answer.upper()=="B" or user_answer.upper()=="C":
                        break
                    else:
                        print("Please enter a valid answer (A, B, or C)")
                if user_answer.upper()==correct_answer.upper():
                    # User answer is checked against the fetched correct answer, score incremented if so.
                    correct_ct+=1
                    print("Correct.\n")
                else:
                    print("Incorrect.\n")
                # Question data is stored for retrieval by write_results.
                used_questions.append(question_info)
                student_answers.append(user_answer.upper())
                question_number+=1
                # Grabs the current time and compares it against the time limit of 600 seconds (10 min)
                # If time limit is exceeded, test is terminated.
                cur_time=round(time.time(),0)
                elapsed_time=cur_time-start_time
                if elapsed_time>600 and question_number<test_cap:
                    print("Time limit reached. Test is concluding early.")
                    skipped=test_cap-question_number
                    print("Number of questions remaining: "+str(skipped)+"\n")
                    break
            # Final results are calculated and all necessary values are passed to write_results
            score=correct_ct*weight
            endtime=round(time.time(),0)
            ttime=endtime-start_time
            write_results(user_info,score,ttime,used_questions,student_answers,skipped)
            break
        print("Quiz has ended. Please enter (Q) to exit the program or (S) to start a new quiz.")
        while True:
            choice=input()
            if choice.upper()=="Q" or choice.upper()=="S":
                break
            else:
                print("Valid entries are 'Q' and 'S'")
        if choice.upper()=="Q":
            # Program reaches end here.
            print("Program has been terminated.")
            break
        elif choice.upper()=="S":
            # Uses imported clear_output function to wipe the previous student's test from the screen.
            # Test program returns to beginning and all values are reinitialized.
            clear_output()
            print("Starting over for a new student.")
            continue
            
if __name__=="__main__":
    main()

Starting over for a new student.


First name:		 Garrett
Last name:		 Jones
Student ID (A#####):	 A23456



How many questions would you like your quiz to have?
(A) - 10 questions
(B) - 20 questions


 a


Proceeding with 10 questions

Question #1: What does print(file.read((10))
A: read the first 10 lines
B: read the first 10 characters
C: read the first 10 words


...

Incorrect.

[Skipping duplicate question]
Question #10: Which of these is not a real python package?
A: pandas
B: iostream
C: numpy


Answer:	 a


Incorrect.

Results successfully written to Garrett_Jones_A23456.txt
Quiz has ended. Please enter (Q) to exit the program or (S) to start a new quiz.


 q


Program has been terminated.


2
